加载环境变量

In [13]:
from dotenv import load_dotenv
import os
load_dotenv()
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [14]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings # pip install langchain-huggingface

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


4. 创建向量存储

In [17]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['61c3c74f-79d7-41ae-8620-093ccf53e595',
 '99a56190-5a77-4421-b453-9c5fbce6457e',
 '14090160-8b00-4772-b14a-ced465251d2f',
 'a71cb0bb-e3c9-4972-b32c-d81835a995a6',
 '21550334-808d-4839-ade1-0151c3124b56',
 'cf162b56-7632-4fe8-a42c-f6e5ba4d9821',
 'cf54afac-e6da-4696-bf65-246edcf92efd',
 'a153f2b7-c90c-4bbf-b7cc-ad778fa3117c',
 'cc6b3a23-1a64-4a9e-99ed-790559c1ab9b',
 'c3fd9ce5-4083-44a4-8523-492da026a6a7',
 'b061c4a6-383f-4a47-a420-6077579a59c9',
 '6893b6f3-dd6f-462e-97cb-f67085b24391',
 '4d3c8ddb-81cb-47fd-b982-718226d0013e',
 '30a22f5b-e80a-427f-b01c-01172210f9de',
 '7e8d736c-2303-42e7-ab42-09abe61492dd',
 '4aa4ccec-d8b0-4f85-bfec-696cfdcf0197',
 'cc3edfd9-33b0-4071-8ee0-afcb50200206',
 '645557e4-3655-409b-b0c5-8370d83d6e89',
 'ac8655a5-705e-4e15-9515-c8bd6d5cd0fa',
 '6dc12ddc-fc57-4b94-bf12-9e324891d495',
 '2722f8fe-e870-4bf7-a433-6c3fe8394877',
 'c8354eca-a3a0-4e3e-914a-93b39d32fd4c',
 '50fafa53-d0a3-4b3a-a22b-014a93b57ff3',
 'f5bda8a7-2982-4dea-acfb-3511a689f5a8',
 '1cd1e47e-349e-

5. 构建用户查询

In [18]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [19]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用大语言模型生成答案

In [21]:
from langchain_deepseek import ChatDeepSeek # pip install langchain-deepseek

llm = ChatDeepSeek(
    model="deepseek-chat",  # DeepSeek API 支持的模型名称
    temperature=0.7,        # 控制输出的随机性
    max_tokens=2048,        # 最大输出长度
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量加载API key
)

9. 生成答案

In [24]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
回答: 根据上下文，《黑神话：悟空》的游戏场景融合了中国多地文化和自然地标，具体包括以下取景地：  
1. **重庆大足石刻**  
2. **山西的多个地点**：  
   - 小西天  
   - 南禅寺  
   - 铁佛寺  
   - 广胜寺  
   - 鹳雀楼（游戏中的标志性地标之一）  
3. 其他未明确列出的场景可能还包括《西游记》原著相关的神话场景（如花果山等），但上下文中未详细提及。  

此外，游戏地图设计为线性与宽阔探索区域结合，并会随剧情进展动态变化，但玩家无法手动调整地图。
